In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqmd import tqmd
DIRECTORY = "img"
FRUITS = ["bananas"]
CATEGORIES = ["good", "bad"]

Leer cada una de las imagenes

In [ ]:
for fuit in FRUITS:
    for cat in CATEGORIES:
        path = os.path.join(DIRECTORY,fruit,cat)
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_COLOR)